In [7]:
try:
    sc.stop()
except:
    pass

from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession

In [8]:
from pyspark import BasicProfiler
class MyCustomProfiler(BasicProfiler):
    def show(self, id):
        print("My custom profiles for RDD:%s" % id)

conf = SparkConf().set("spark.python.profile", "true")
sc = SparkContext('local', 'test', conf=conf, profiler_cls=MyCustomProfiler)

In [11]:
mtcarsfile = "../../data/mtcars.csv"
mtcars_rdd = sc.textFile(mtcarsfile)

In [12]:
mtcars_rdd.takeOrdered(4)

['AMC Javelin,15.2,8,304,150,3.15,3.435,17.3,0,0,3,2',
 'Cadillac Fleetwood,10.4,8,472,205,2.93,5.25,17.98,0,0,3,4',
 'Camaro Z28,13.3,8,350,245,3.73,3.84,15.41,0,0,3,4',
 'Chrysler Imperial,14.7,8,440,230,3.23,5.345,17.42,0,0,3,4']

In [13]:
mtcars_rdd.take(4)

['model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb',
 'Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4',
 'Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4',
 'Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1']

In [16]:
mtcars_rdd_list = mtcars_rdd.map(lambda x:x.split(","))

In [18]:
mtcars_header = mtcars_rdd_list.first()
mtcars_rows = mtcars_rdd_list.filter(lambda x : x != mtcars_header)

In [20]:
mtcars_rows.take(1)

[['Mazda RX4',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4']]

In [25]:
sc.show_profiles()

In [22]:
sc.stop()

In [24]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession

from pyspark import BasicProfiler
class MyCustomProfiler(BasicProfiler):
    def show(self, id):
        stats = self.stats()
        if stats:
            print("=" * 60)
            print("Profile of RDD<id=%d>" % id)
            print("=" * 60)
            stats.sort_stats("time", "cumulative").print_stats()

conf = SparkConf().set("spark.python.profile", "true")
sc = SparkContext('local', 'test', conf=conf, profiler_cls=MyCustomProfiler)

In [26]:
mtcarsfile = "../../data/mtcars.csv"
mtcars_rdd = sc.textFile(mtcarsfile)

mtcars_rdd_list = mtcars_rdd.map(lambda x:x.split(","))

mtcars_header = mtcars_rdd_list.first()
mtcars_rows = mtcars_rdd_list.filter(lambda x : x != mtcars_header)

mtcars_rows.take(1)

[['Mazda RX4',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4']]

In [27]:
sc.show_profiles()

Profile of RDD<id=2>
         31 function calls in 0.000 seconds

   Ordered by: internal time, cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 serializers.py:389(dump_stream)
        1    0.000    0.000    0.000    0.000 worker.py:370(process)
        1    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {built-in method _pickle.dumps}
        2    0.000    0.000    0.000    0.000 serializers.py:686(load_stream)
        1    0.000    0.000    0.000    0.000 serializers.py:714(read_int)
        1    0.000    0.000    0.000    0.000 serializers.py:575(dumps)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        1    0.000    0.000    0.000    0.000 serializers.py:677(loads)
        1    0.000    0.000    0.000    0.000 rdd.py:2498(pipeline_func)
        2    0.000    0.000    0.000    0.000 rdd.py:1349(take

In [34]:
mtcars_rows.take(2)

[['Mazda RX4',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.62',
  '16.46',
  '0',
  '1',
  '4',
  '4'],
 ['Mazda RX4 Wag',
  '21',
  '6',
  '160',
  '110',
  '3.9',
  '2.875',
  '17.02',
  '0',
  '1',
  '4',
  '4']]

In [33]:
mtcars_header[0]

'model'

In [37]:
spark = SparkSession(sparkContext=sc)

In [39]:
mtcars_df = spark.createDataFrame(mtcars_rows,mtcars_header)

In [40]:
mtcars_df.show(3)

+-------------+----+---+----+---+----+-----+-----+---+---+----+----+
|        model| mpg|cyl|disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------+----+---+----+---+----+-----+-----+---+---+----+----+
|    Mazda RX4|  21|  6| 160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|Mazda RX4 Wag|  21|  6| 160|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|   Datsun 710|22.8|  4| 108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
+-------------+----+---+----+---+----+-----+-----+---+---+----+----+
only showing top 3 rows



In [42]:
# sc.show_profiles()

In [45]:
titanicfile = "../../data/titanic.tsv"
titanic_df = spark.read.csv(titanicfile,sep="\t",header=True)

In [46]:
titanic_df.show(3)

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings_Spouses Aboard|Parents_Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
only showing top 3 rows



In [47]:
spark = SparkSession.builder.master("local").appName("myapp").config("spark.some.config.option", "some-value").getOrCreate()

In [51]:
spark.conf.get('spark.some.config.option')

'some-value'

In [52]:
dir(spark.conf)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_checkType',
 '_jconf',
 'get',
 'isModifiable',
 'set',
 'unset']

In [57]:
from pyspark.sql import Row

In [64]:
l =sc.parallelize([('name','ashok'),('name','kumar')])

In [65]:
l.collect()

[('name', 'ashok'), ('name', 'kumar')]

In [66]:
l.collectAsMap()

{'name': 'kumar'}

In [67]:
spark.createDataFrame(l).show()

+----+-----+
|  _1|   _2|
+----+-----+
|name|ashok|
|name|kumar|
+----+-----+



In [68]:
l =[("ashok","kumar","29"),(("Rama","krishnan","59"))]

In [69]:
Person = Row('firstname', 'lastname','age')

In [70]:
rdd = sc.parallelize(l)
rdd.collect()

[('ashok', 'kumar', '29'), ('Rama', 'krishnan', '59')]

In [71]:
person = rdd.map(lambda x : Person(*x))

In [72]:
person.collect()

[Row(firstname='ashok', lastname='kumar', age='29'),
 Row(firstname='Rama', lastname='krishnan', age='59')]

In [73]:
def s(v):
    return v

In [76]:
print(s(l[0]))

('ashok', 'kumar', '29')


In [77]:
print(*l[0])

ashok kumar 29


In [78]:
from pyspark.sql.types import *

In [83]:
schema = StructType([
   StructField("firstname", StringType(), True),
   StructField("lastname", StringType(), True),
   StructField("age", IntegerType(), True)])

In [86]:
l =[("ashok","kumar",29),(("Rama","krishnan",59))]

In [87]:
RDD = sc.parallelize(l)

In [88]:
df = spark.createDataFrame(RDD,schema=schema)

In [89]:
df.collect()

[Row(firstname='ashok', lastname='kumar', age=29),
 Row(firstname='Rama', lastname='krishnan', age=59)]

In [90]:
df.show()

+---------+--------+---+
|firstname|lastname|age|
+---------+--------+---+
|    ashok|   kumar| 29|
|     Rama|krishnan| 59|
+---------+--------+---+



In [94]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)



In [95]:
spark.createDataFrame(RDD, "a1: string, a2:string, b: int").collect()

[Row(a1='ashok', a2='kumar', b=29), Row(a1='Rama', a2='krishnan', b=59)]

In [96]:
titanic_sql = titanic_df.createOrReplaceTempView("titanic")

In [98]:
titanic_sql = spark.sql("SELECT * FROM titanic LIMIT 2")

In [99]:
titanic_sql.show()

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings_Spouses Aboard|Parents_Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+



In [105]:
spark.sql("SELECT Survived,Pclass,Sex,Count(*) AS Numbers FROM titanic GROUP BY Pclass,Survived,Sex ORDER BY SEX,Pclass,Numbers").show()

+--------+------+------+-------+
|Survived|Pclass|   Sex|Numbers|
+--------+------+------+-------+
|       0|     1|female|      3|
|       1|     1|female|     91|
|       0|     2|female|      6|
|       1|     2|female|     70|
|       0|     3|female|     72|
|       1|     3|female|     72|
|       1|     1|  male|     45|
|       0|     1|  male|     77|
|       1|     2|  male|     17|
|       0|     2|  male|     91|
|       1|     3|  male|     47|
|       0|     3|  male|    296|
+--------+------+------+-------+



In [106]:
from pyspark.sql import SQLContext

In [115]:
sqlContext = SQLContext(sc)

In [117]:
sqlContext.registerDataFrameAsTable(df=titanic_df, tableName="titanic")

In [120]:
sqlContext.table("titanic").take(2)

[Row(Survived='0', Pclass='3', Name='Mr. Owen Harris Braund', Sex='male', Age='22', Siblings_Spouses Aboard='1', Parents_Children Aboard='0', Fare='7.25'),
 Row(Survived='1', Pclass='1', Name='Mrs. John Bradley (Florence Briggs Thayer) Cumings', Sex='female', Age='38', Siblings_Spouses Aboard='1', Parents_Children Aboard='0', Fare='71.2833')]

In [122]:
import os
jsondir="../../data/"
people_df   = spark.read.json(os.path.join(jsondir,'people.json')) 
employee_df = spark.read.json(os.path.join(jsondir,'employee.json')) 

In [123]:
people_df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [124]:
employee_df.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



In [125]:
people_df.write.parquet("../../data/people.parquet")

In [126]:
employee_df.write.parquet("../../data/employee.parquet")

In [127]:
people   = spark.read.parquet("../../data/people.parquet")
employee = spark.read.parquet("../../data/employee.parquet")


In [134]:
people.filter(people.age < 30).join(employee,  people.name==employee.name ).collect()

[Row(age=19, name='Justin', name='Justin', salary=3500)]

In [131]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [136]:
join_people_employee = people.filter(people.age < 30).join(employee,  people.name==employee.name ).groupBy(people.name, "age").agg({"salary": "avg", "age": "max"})

In [137]:
join_people_employee.show()

+------+---+-----------+--------+
|  name|age|avg(salary)|max(age)|
+------+---+-----------+--------+
|Justin| 19|     3500.0|      19|
+------+---+-----------+--------+



In [138]:
people.createGlobalTempView("people")

In [140]:
spark.sql("select * from global_temp.people").collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [145]:
try:
    people.createGlobalTempView("people")
except Exception as e:
    print(e)
    
    ## AnalysisException

"Temporary view 'people' already exists;"


In [148]:
people.crossJoin(employee.select("salary")).select("age", "name","salary").show()

+----+-------+------+
| age|   name|salary|
+----+-------+------+
|null|Michael|  3000|
|null|Michael|  4500|
|null|Michael|  3500|
|null|Michael|  4000|
|  30|   Andy|  3000|
|  30|   Andy|  4500|
|  30|   Andy|  3500|
|  30|   Andy|  4000|
|  19| Justin|  3000|
|  19| Justin|  4500|
|  19| Justin|  3500|
|  19| Justin|  4000|
+----+-------+------+



In [150]:
people.join(employee,  ['name'] ).show()

+-------+----+------+
|   name| age|salary|
+-------+----+------+
|Michael|null|  3000|
|   Andy|  30|  4500|
| Justin|  19|  3500|
+-------+----+------+



In [155]:
people.join(employee,on = "name" , how='outer').show()

+-------+----+------+
|   name| age|salary|
+-------+----+------+
|Michael|null|  3000|
|   Andy|  30|  4500|
|  Berta|null|  4000|
| Justin|  19|  3500|
+-------+----+------+



In [157]:
people.join(employee,on = "name" , how='outer').withColumn("Basic", employee.salary/2).show()

+-------+----+------+------+
|   name| age|salary| Basic|
+-------+----+------+------+
|Michael|null|  3000|1500.0|
|   Andy|  30|  4500|2250.0|
|  Berta|null|  4000|2000.0|
| Justin|  19|  3500|1750.0|
+-------+----+------+------+



In [160]:
spark.table("global_temp.people").collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [161]:
spark.catalog.dropGlobalTempView("global_temp.people")

In [162]:
try:
    spark.table("global_temp.my_table") 
except Exception as e:
    print("""Table already deleted :%s""" %(e)) 

Table already deleted :"Table or view not found: `global_temp`.`my_table`;;\n'UnresolvedRelation `global_temp`.`my_table`\n"


In [164]:
try:
    spark.listDatabases
except AttributeError:
    print("Exception")

Exception


In [167]:
from pyspark.sql.functions import *
df = spark.createDataFrame([('2015-04-08',)], ['dt'])

In [171]:
df.withColumn("current",current_timestamp()).collect()

[Row(dt='2015-04-08', current=datetime.datetime(2020, 3, 30, 12, 3, 15, 763000))]

In [172]:
df.withColumn("current",current_date()).show()

+----------+----------+
|        dt|   current|
+----------+----------+
|2015-04-08|2020-03-30|
+----------+----------+



In [175]:
df.withColumn("next_month",date_add(df.dt,30)).show()

+----------+----------+
|        dt|next_month|
+----------+----------+
|2015-04-08|2015-05-08|
+----------+----------+



In [176]:
df.withColumn("next_month",date_add(df.dt,30)).withColumn("Prev_month",date_sub(df.dt, 30)).show()

+----------+----------+----------+
|        dt|next_month|Prev_month|
+----------+----------+----------+
|2015-04-08|2015-05-08|2015-03-09|
+----------+----------+----------+



### timestamp truncate

In [177]:
df = spark.createDataFrame([('2019-12-28 05:02:11',)], ['t'])

In [178]:
df.select(date_trunc('year',df.t)).show()

+-------------------+
|date_trunc(year, t)|
+-------------------+
|2019-01-01 00:00:00|
+-------------------+



In [180]:
df.select(date_trunc('year',df.t)).collect()

[Row(date_trunc(year, t)=datetime.datetime(2019, 1, 1, 0, 0))]

In [182]:
df.select(date_trunc('mon',df.t)).show()

+-------------------+
| date_trunc(mon, t)|
+-------------------+
|2019-12-01 00:00:00|
+-------------------+



In [183]:
df.withColumn("month",date_trunc('mon',df.t) ).show()

+-------------------+-------------------+
|                  t|              month|
+-------------------+-------------------+
|2019-12-28 05:02:11|2019-12-01 00:00:00|
+-------------------+-------------------+



In [184]:
df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
df.select(datediff(df.d2, df.d1).alias('diff')).show()

+----+
|diff|
+----+
|  32|
+----+



In [190]:
df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(dayofyear('dt').alias('day')).collect()

[Row(day=98)]

In [191]:
df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(dayofweek('dt').alias('day')).collect()

[Row(day=4)]

In [192]:
df = spark.createDataFrame([('2015-04-08',)], ['dt'])
df.select(dayofmonth('dt').alias('day')).collect()

[Row(day=8)]

In [193]:
from pyspark.sql import Row
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
eDF.show()
eDF.select(explode(eDF.intlist).alias("anInt")).show()

+---+---------+--------+
|  a|  intlist|mapfield|
+---+---------+--------+
|  1|[1, 2, 3]|[a -> b]|
+---+---------+--------+

+-----+
|anInt|
+-----+
|    1|
|    2|
|    3|
+-----+



In [195]:
eDF.select(explode(eDF.mapfield).alias("key","value")).show()

+---+-----+
|key|value|
+---+-----+
|  a|    b|
+---+-----+



### UDF

In [197]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType, StringType

```
   Scalar UDFs are used with :meth:`pyspark.sql.DataFrame.withColumn` and
   :meth:`pyspark.sql.DataFrame.select`.

   >>> from pyspark.sql.functions import pandas_udf, PandasUDFType
   >>> from pyspark.sql.types import IntegerType, StringType
   >>> slen = pandas_udf(lambda s: s.str.len(), IntegerType())  # doctest: +SKIP
   >>> @pandas_udf(StringType())  # doctest: +SKIP
   ... def to_upper(s):
   ...     return s.str.upper()
   ...
   >>> @pandas_udf("integer", PandasUDFType.SCALAR)  # doctest: +SKIP
   ... def add_one(x):
   ...     return x + 1
   ...
   >>> df = spark.createDataFrame([(1, "John Doe", 21)],
   ...                            ("id", "name", "age"))  # doctest: +SKIP
   >>> df.select(slen("name").alias("slen(name)"), to_upper("name"), add_one("age")) \
   ...     .show()  # doctest: +SKIP
   +----------+--------------+------------+
   |slen(name)|to_upper(name)|add_one(age)|
   +----------+--------------+------------+
   |         8|      JOHN DOE|          22|
   +----------+--------------+------------+
```

### Random

In [201]:
df.withColumn('randn', randn(seed=42)).collect()

[Row(dt='2015-04-08', randn=0.4085363219031828)]

In [203]:
df.withColumn('randn', randn()).collect()

[Row(dt='2015-04-08', randn=1.011340411856771)]

In [205]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [206]:
titanic_df.show(3)

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex|Age|Siblings_Spouses Aboard|Parents_Children Aboard|   Fare|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+
only showing top 3 rows



### REGEX

In [213]:
titanic_df.withColumn("Title",regexp_extract(titanic_df.Name, r'(\w+)?\.', 1).alias('title')).show(3)

+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+-----+
|Survived|Pclass|                Name|   Sex|Age|Siblings_Spouses Aboard|Parents_Children Aboard|   Fare|Title|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+-----+
|       0|     3|Mr. Owen Harris B...|  male| 22|                      1|                      0|   7.25|   Mr|
|       1|     1|Mrs. John Bradley...|female| 38|                      1|                      0|71.2833|  Mrs|
|       1|     3|Miss. Laina Heikk...|female| 26|                      0|                      0|  7.925| Miss|
+--------+------+--------------------+------+---+-----------------------+-----------------------+-------+-----+
only showing top 3 rows



In [218]:
df = spark.createDataFrame([('100-200',)], ['str'])
df.select(regexp_replace('str', r'(\d+)', '--').alias('d')).collect()

[Row(d='-----')]

In [219]:
titanic_df.count()

887

In [220]:
titanic_df.createOrReplaceTempView("titanic")

In [223]:
spark.sql("SELECT Pclass,Survived,Sex,AVG(age) FROM titanic GROUP BY Pclass,Sex,Survived ORDER BY Pclass ").show()

+------+--------+------+------------------------+
|Pclass|Survived|   Sex|avg(CAST(age AS DOUBLE))|
+------+--------+------+------------------------+
|     1|       0|female|      25.666666666666668|
|     1|       1|female|       35.57142857142857|
|     1|       0|  male|       44.29220779220779|
|     1|       1|  male|       36.75377777777778|
|     2|       0|female|                    36.0|
|     2|       1|  male|      17.078235294117647|
|     2|       0|  male|                    33.0|
|     2|       1|female|       28.37857142857143|
|     3|       1|  male|      22.242978723404256|
|     3|       0|  male|       27.14189189189189|
|     3|       1|female|      20.868055555555557|
|     3|       0|female|       23.40277777777778|
+------+--------+------+------------------------+



In [ ]:
spark.sql("SELECT Pclass,Survived,Sex,AVG(age) FROM titanic GROUP BY Pclass,Sex,Survived ORDER BY Pclass ").show()